# Benchmarking

## Import Libraries

In [ ]:
import os
os.chdir('..')

from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.reporting import formatter as fmt, plots
import statsmodels.api as sm

## Import bmk data

In [ ]:
bmk_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\bmk_data.xlsx', sheet_name='data_m', index_col=0)
bmk_df = bmk_df[['HFRXM Index','HFRIMI Index',
                 'SGSLRPEW Index', 'SGSLRPWH Index', 'SGIXTFXA Index', 'NEIXCTAT Index',
                 'HFRXAR Index', 'HFRXEH Index', 'HFRXEW Index', 'HFRXED Index',
                 'HFRXCA Index', 'HFRXEMC Index', 'HFRXCOM Index',
                 'SECTGLMA Index', 'SECTNEUT Index','SECTMSTR Index',
                 'HEDGGLMA Index', 'HEDGNAV Index','HEDGNEUT Index','HEDGCONV Index',
                 'CSLAB Index', 'CSLABED Index', 'CSLABGS Index', 'CSLABMF Index',
                 'CSLABT15 Index', 'CSLABT18 Index', 'CSLABT20 Index']]
#rename columns
bmk_df.columns = ['HFRX Macro/CTA', 'HFRI Macro Total Index',
                  'Eqty Risk Prem', 'Eqty Risk Prem-Hedged','SGI Cross Asset Trend', 'SG Trend Index',
                  'HFRX Abs Ret', 'HFRX Eqty Hedge', 'HFRX Equal Wgtd', 'HFRX Event Driven',
                  'HFRX RV FI Conv Arb', 'HFRX EM', 'HFRX Macro Comdty',
                  'CS AllHedge Global Macro', 'CS AllHedge Eqty Mkt Neutral','CS AllHedge Multi_Strat',
                  'CS Global Macro', 'CS Hedge Fund', 'CS Eqty Mkt Neutral', 'CS Conv Arb',
                 'CS Liq Alt Beta', 'CS Event Driven', 'CS Global Strat', 'CS Managed Fut',
                 'CS Managed Fut-15% vol','CS Managed Fut-18% vol', 'CS Managed Fut-20% vol']

#format data to returns
bmk_df = dm.format_data(bmk_df)


## Display bmk Returns

In [ ]:
fmt.get_returns_styler(bmk_df)

## Import liq bmk data

In [ ]:
liq_alts_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\mgr_data_m.xlsx', sheet_name='bny',index_col=0)
liq_alts_sp = liq_alts_df[['Total Liquid Alts',  'Global Macro',  'Trend Following','Alt Risk Premia']]
liq_alts_sp = liq_alts_sp.iloc[52:,]
liq_alts_mgr = liq_alts_df[['1907 ARP EM', '1907 ARP TF', '1907 CFM ISE Risk Premia','1907 Campbell TF', '1907 III CV',
                            '1907 III Class A', '1907 Kepos RP','1907 Penso Class A', '1907 Systematica TF',
                            'ABC Reversion','Acadian Commodity AR', 'Black Pearl RP','Blueshift', 'Bridgewater Alpha',
                            'DE Shaw Oculus Fund', 'Duality','Element Capital', 'Elliott', 'One River Trend']]

## Display Liq Alts Returns

In [ ]:
@interact
def display_liq_alts_returns(stats=['Sub Portfolios', 'Managers']):
    if stats == 'Sub Portfolios':
        return fmt.get_returns_styler(liq_alts_sp)
    if stats == 'Managers':
        return fmt.get_returns_styler(liq_alts_mgr)

# Sub Portfolios Regressions
## Multiple Linear Regression With statsmodels

In [ ]:
sp_results_dict = {}

for col in liq_alts_sp.columns:
    
    data = dm.merge_data_frames(liq_alts_sp[[col]].copy(),bmk_df)
    x = data[list(bmk_df.columns)].copy()
    x = x.to_numpy()
   
    y = data[[col]].copy()
    y.dropna(inplace=True)
    y = np.array(list(y[col]))
    
    x, y = np.array(x), np.array(y)
    x = sm.add_constant(x)
    
    model = sm.OLS(y, x)
    sp_results_dict[col] = model.fit()


## Display Regression Results

In [ ]:
@interact
def display_sp_reg_results(sub_portfolio = sp_results_dict.keys()):
    return print(sp_results_dict[sub_portfolio].summary())


## Display Predictor Keys

In [ ]:
bmk_keys ={}
bmk_len = len(bmk_df.columns)
for n in range(len(bmk_df.columns)):
    print('x{}: {}'.format(n+1,bmk_df.columns[n]))

# Manager Regressions
## Multiple Linear Regression With statsmodels

In [ ]:
mgr_results_dict = {}

for col in liq_alts_mgr.columns:
    
    data = dm.merge_data_frames(liq_alts_mgr[[col]].copy(),bmk_df)
    x = data[list(bmk_df.columns)].copy()
    x = x.to_numpy()
   
    y = data[[col]].copy()
    y.dropna(inplace=True)
    y = np.array(list(y[col]))
    
    x, y = np.array(x), np.array(y)
    x = sm.add_constant(x)
    
    model = sm.OLS(y, x)
    mgr_results_dict[col] = model.fit()


## Display Regression Results

In [ ]:
@interact
def display_mgr_reg_results(manager = mgr_results_dict.keys()):
    return print(mgr_results_dict[manager].summary())


## Display Predictor Keys

In [ ]:
bmk_keys ={}
bmk_len = len(bmk_df.columns)
for n in range(len(bmk_df.columns)):
    print('x{}: {}'.format(n+1,bmk_df.columns[n]))